In [ ]:
# Welcome to my first summarization chat bot

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
ed = Website("https://cnn.com")
print(ed.title)
print(ed.text)

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown. When you give response to user, please use some emoji to make the response fun!"

In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(ed))

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(ed)

In [ ]:
def summarize(url):
    website = Website(url)
    payload = {
        "model": MODEL,
        "messages": messages_for(ed),
        "stream": False
    }
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    return response.json()['message']['content']

In [ ]:
!ollama pull llama3.2

In [ ]:
summarize("https://edwarddonner.com")

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages_for(ed))
print(response['message']['content'])

In [ ]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages_for(ed)
)

print(response.choices[0].message.content)